In [15]:
import pandas as pd

In [16]:
version = "V4"
year = "2024"

form = "2"
rev = "_rev2"

# company_sym = "BAM"
# company_sym = "BAY"
# company_sym = "CIMBT"
# company_sym = "LHFG"
# company_sym = "SCB"

# company_sym = "ACAP"
company_sym = "AEONTS"
# company_sym = "JMT"
# company_sym = "SAWAD"

# company_sym = "BYD"
# company_sym = "KGI"
# company_sym = "MFC"

# company_sym = "BUI"
# company_sym = "NKI"
# company_sym = "TIPH"
# company_sym = "TSI"

# company_sym = "3BBIF"
# company_sym = "BKKCP"
# company_sym = "BTSGIF"
# company_sym = "FTREIT"
# company_sym = "MJLF"

prod_result_path = f"./Listed_Mapping_Prod/{year}/{company_sym}_Q2_{year}.csv"
ai_result_path = f"./AI_Mapping_Result/{version}/{form}_{company_sym}_mapping_result_{version}{rev}.csv"

output_csv_path = f"./Compare_Result/{form}_{company_sym}_{version}{rev}_compare.csv"
output_txt_path = f"./Compare_Result/{form}_{company_sym}_{version}{rev}_compare.txt"

print(prod_result_path)
print(ai_result_path)
print(output_csv_path)
print(output_txt_path)

./Listed_Mapping_Prod/2024/AEONTS_Q2_2024.csv
./AI_Mapping_Result/V4/2_AEONTS_mapping_result_V4_rev2.csv
./Compare_Result/2_AEONTS_V4_rev2_compare.csv
./Compare_Result/2_AEONTS_V4_rev2_compare.txt


In [17]:
th_month = [
    'มกราคม', 
    'กุมภาพันธ์', 
    'มีนาคม',
    'เมษายน',
    'พฤษภาคม',
    'มิถุนายน',
    'กรกฎาคม',
    'สิงหาคม',
    'กันยายน',
    'ตุลาคม',
    'พฤศจิกายน',
    'ธันวาคม',
]

th_month_short = [
    'ม.ค.', 
    'ก.พ.', 
    'มี.ค.',
    'เม.ย.',
    'พ.ค.',
    'มิ.ย.',
    'ก.ค.',
    'ส.ค.',
    'ก.ย.',
    'ต.ค.',
    'พ.ย.',
    'ธ.ค.',
]

In [18]:
def replace_comma(name):
    return name.replace(",", "")

def replace_month(name):
    for month in th_month:
        if (month in name):
            name = name.replace(month, "")

    for month in th_month_short:
        if (month in name):
            name = name.replace(month, "")
    return name

In [19]:
dtype = {
    'query': 'string',
    'set_code': 'string'
}

ai_df = pd.read_csv(ai_result_path, encoding='utf-8', dtype=dtype)

ai_df_query = ai_df['query']
ai_df_query.replace(to_replace='"', value='', inplace=True)
ai_df_query.replace(to_replace=' ', value='', regex=True, inplace=True)
ai_df_query.replace(to_replace=r'[0-9]', value='', regex=True, inplace=True)
ai_df['query'] = ai_df_query

ai_df_query = ai_df['query']
for index, row in ai_df_query.items():
    if (pd.isna(row)):
        continue
    name = row
    name_replace = replace_month(name)
    name_replace = replace_comma(name_replace)
    ai_df['query'].iloc[index] = name_replace

ai_df.iloc[38]['query']

'หนี้สินหมุนเวียนอื่น'

In [20]:
dtype = {
    'account_name': 'string',
    'account_code': 'string',
    'parent_account_code': 'string',
}

prod_df = pd.read_csv(prod_result_path, encoding='utf-8', dtype=dtype)
prod_acc_name = prod_df['account_name']
prod_acc_name.replace(to_replace=r'\{{\w+\}}', value='', regex=True, inplace=True)
prod_df['account_name'] = prod_acc_name
# prod_df.iloc[22]

In [21]:
compared_list = []

for index, row in prod_df.iterrows():
    prod_taxo_name = row['taxonomy_account_name']
    prod_acc_code = row['account_code']
    prod_parent_acc_code = row['parent_account_code']
    prod_acc_name = row['account_name']
    prod_values = row['values']
    # print(prod_acc_name)
    
    ai_row = ai_df[(ai_df['set_code'] == prod_acc_code) | (ai_df['set_code'] == prod_parent_acc_code)]
    # ai_row = ai_df[(ai_df['query'] == prod_acc_name) & ((ai_df['set_code'] == prod_acc_code) | (ai_df['set_code'] == prod_parent_acc_code))]

    select_index = None
    for i, r in ai_row.iterrows():
        if (prod_acc_name in r['query']):
            select_index = i
            break

    ai_acc_code = ""
    ai_acc_name = ""
    unmatch_remark = ""
    if (len(ai_row.index) > 0 and select_index is not None):
        ai_acc_code = ai_df['set_code'].iloc[select_index]
        ai_acc_name = ai_df['query'].iloc[select_index]
    else:
        select_df = ai_df[ai_df['query'] == prod_acc_name]
        for i, r in select_df.iterrows():
            unmatch_remark += f"{r['set_code']}-{r['query']}-{r['item_in_taxonomy']}-{r['reasoning']} | "
            
        if (unmatch_remark == ""):
            print(f"Not Found {prod_acc_code} in extarct result")
        else:
            print(f"Not found {prod_acc_code} in AI mapping")

    compare_item = {
        "taxonomy_account_name": prod_taxo_name,
        "ai_account_code": ai_acc_code,
        "ai_acc_name": ai_acc_name,
        "prod_account_name": prod_acc_name,
        "prod_account_code": prod_acc_code,
        "prod_parent_account_code": prod_parent_acc_code,
        "is_match": str(ai_acc_code) == str(prod_acc_code),
        "is_match_parent": str(ai_acc_code) == str(prod_parent_acc_code),
        "unmatch_remark": unmatch_remark
    }

    compared_list.append(compare_item)

Not found 122040 in AI mapping
Not found 123140 in AI mapping
Not found 122400 in AI mapping
Not found 123000 in AI mapping
Not Found 210500 in extarct result
Not found 211890 in AI mapping
Not found 212100 in AI mapping
Not found 212400 in AI mapping
Not found 310120 in AI mapping
Not found 439790 in AI mapping
Not Found 440500 in extarct result
Not found 439790 in AI mapping
Not found 430900 in AI mapping
Not Found 440500 in extarct result
Not found 514900 in AI mapping
Not found 510400 in AI mapping
Not found 511700 in AI mapping
Not found 511500 in AI mapping
Not found 514910 in AI mapping
Not found 510700 in AI mapping
Not found 511610 in AI mapping
Not Found 510800 in extarct result
Not Found 515000 in extarct result
Not found 514800 in AI mapping
Not found 519900 in AI mapping
Not found 510700 in AI mapping
Not found 529900 in AI mapping
Not Found 521200 in extarct result
Not Found 531800 in extarct result
Not found 530700 in AI mapping
Not found 530800 in AI mapping
Not found 5

In [22]:
count_unmatch = 0
count_match_parent = 0
count_not_extract = 0
count_no_code = 0

unmatch_list = []
for item in compared_list:
    if (item['is_match'] == False):
        if (item['is_match_parent'] == True):
            print("[1] Match Parent")
            count_match_parent += 1
        elif (item['unmatch_remark'] == ""):
            print("[2] Probably unextact item from financial statement")
            count_not_extract += 1
        elif (item['ai_account_code'] == "" and item['ai_acc_name'] == ""):
            print("[3] Not match accode and name with AI")
            count_no_code += 1
            
        count_unmatch += 1
        print(item)
        unmatch_list.append(item)
        print("\n")

print(f"Total Unmatch: {count_unmatch}")
print(f"Total Match Parent: {count_match_parent}")
print(f"Total Not Extract: {count_not_extract}")
print(f"Total No Candiate: {count_no_code}")

[1] Match Parent
{'taxonomy_account_name': 'สินทรัพย์หมุนเวียนอื่น', 'ai_account_code': '119900', 'ai_acc_name': 'สินทรัพย์หมุนเวียนอื่น', 'prod_account_name': 'สินทรัพย์หมุนเวียนอื่น', 'prod_account_code': '119990', 'prod_parent_account_code': '119900', 'is_match': False, 'is_match_parent': True, 'unmatch_remark': ''}


[3] Not match accode and name with AI
{'taxonomy_account_name': 'ลูกหนี้การค้า', 'ai_account_code': '', 'ai_acc_name': '', 'prod_account_name': 'ลูกหนี้การค้า', 'prod_account_code': '122040', 'prod_parent_account_code': '122000', 'is_match': False, 'is_match_parent': False, 'unmatch_remark': '112040-ลูกหนี้การค้า-ลูกหนี้การค้า-The input item name ลูกหนี้การค้า matched with item_in_taxonomy ลูกหนี้การค้า in the Taxonomy list. And the input metadata financial_statement_type งบฐานะการเงิน is matched with financial_statement_type BS_A and fs_category สินทรัพย์ is matched with fs_category สินทรัพย์หมุนเวียน in the Taxonomy list. | 112040-ลูกหนี้การค้า-ลูกหนี้การค้า-Taxonomy

In [23]:
unmatch_df = pd.DataFrame(unmatch_list)
unmatch_df.head()

,taxonomy_account_name,ai_account_code,ai_acc_name,prod_account_name,prod_account_code,prod_parent_account_code,is_match,is_match_parent,unmatch_remark
0,สินทรัพย์หมุนเวียนอื่น,119900,สินทรัพย์หมุนเวียนอื่น,สินทรัพย์หมุนเวียนอื่น,119990,119900,False,True,
1,ลูกหนี้การค้า,,,ลูกหนี้การค้า,122040,122000,False,False,112040-ลูกหนี้การค้า-ลูกหนี้การค้า-The input i...
2,ลูกหนี้อื่น,,,ลูกหนี้อื่น,123140,123100,False,False,113130-ลูกหนี้อื่น-สินทรัพย์ทางการเงินหมุนเวีย...
3,เงินให้กู้ยืมระยะยาวแก่บริษัทย่อย,,,เงินให้กู้ยืมระยะยาวแก่บริษัทย่อย,122400,129999,False,False,122420-เงินให้กู้ยืมระยะยาวแก่บริษัทย่อย-บุคคล...
4,สินทรัพย์อนุพันธ์,,,สินทรัพย์อนุพันธ์,123000,129999,False,False,113000-สินทรัพย์อนุพันธ์-สินทรัพย์อนุพันธ์ - ห...


In [24]:
unmatch_df.to_csv(output_csv_path, encoding='utf-8', index=False, header=True)

### Calculate Accuracy

In [25]:
total_prod_taxonomy = len(prod_df.index)

total_extracted_taxonomy = total_prod_taxonomy - count_not_extract

In [26]:
correct_map = total_extracted_taxonomy - count_no_code
accuracy = (correct_map / total_extracted_taxonomy) * 100
print(f"Accuracy: {accuracy} %")

Accuracy: 83.22147651006712 %


In [27]:
partial_correct_map = correct_map + count_match_parent
accuracy_parent_map = (partial_correct_map / total_extracted_taxonomy) * 100
print(f"Accuracy (include map parent): {accuracy_parent_map} %")

Accuracy (include map parent): 87.91946308724832 %


In [28]:
print(prod_result_path)
print(ai_result_path)

with open(output_txt_path, 'w') as txt_file:
    txt_file.write(f"Reading AI Result from: {ai_result_path}\n")
    txt_file.write(f"Reading Prod Result from: {prod_result_path}\n\n")
    
    txt_file.write(f"Total Taxonomy item from Prod: {total_prod_taxonomy}\n")
    txt_file.write(f"Total Extracted Taxonomy item: {total_extracted_taxonomy}\n")
    txt_file.write(f"Match between Prod and AI (Correct Map): {correct_map}\n")
    txt_file.write(f"Match between Parent Prod and AI (Partialy Correct Map): {count_match_parent}\n")
    txt_file.write(f"Not Match between Prod and AI: {count_unmatch}\n\n")
    
    txt_file.write(f"Accuracy: {accuracy} % \n")
    txt_file.write(f"Accuracy (include parent map): {accuracy_parent_map} % \n\n")
    
    txt_file.write(f"Detail of unmatch item: {output_csv_path} \n\n")

./Listed_Mapping_Prod/2024/AEONTS_Q2_2024.csv
./AI_Mapping_Result/V4/2_AEONTS_mapping_result_V4_rev2.csv
